In [164]:
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
import requests
import pandas as pd
import datetime as dt

In [165]:
# Set up Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Could not get version for google-chrome with the command:  powershell "$ErrorActionPreference='silentlycontinue' ; (Get-Item -Path "$env:PROGRAMFILES\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion ; if (-not $? -or $? -match $error) { (Get-Item -Path "$env:PROGRAMFILES(x86)\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion } if (-not $? -or $? -match $error) { (Get-Item -Path "$env:LOCALAPPDATA\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion } if (-not $? -or $? -match $error) { reg query "HKCU\SOFTWARE\Google\Chrome\BLBeacon" /v version } if (-not $? -or $? -match $error) { reg query "HKLM\SOFTWARE\Wow6432Node\Microsoft\Windows\CurrentVersion\Uninstall\Google Chrome" /v version }"
Current google-chrome version is UNKNOWN
Get LATEST chromedriver version for UNKNOWN google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/100.0.4896.60/chromedriver_win32.zip
Driver has been 

## Mars News

In [ ]:
url = 'https://redplanetscience.com/'
browser.visit(url)

browser.is_element_present_by_css('div.list_text', wait_time=1)

html = browser.html
news_soup = soup(html, 'html.parser')

slide_elem = news_soup.select_one('div.list_text')

slide_elem.find('div', class_='content_title')

news_title = slide_elem.find('div', class_='content_title').get_text()
news_p = slide_elem.find('div', class_='article_teaser_body').get_text()


## Featured Image

In [ ]:
url1 = 'https://spaceimages-mars.com/'
browser.visit(url1)

html1 = browser.html
news_soup1 = soup(html1, 'html.parser')

slide_elem1 = news_soup1.select_one('div.floating_text_area')

image_url_location = slide_elem1.find('a')
image_url= image_url_location['href']
featured_image_url= (f'{url1}{image_url}')
featured_image_url

## Mars Facts

In [ ]:
url2 = 'https://galaxyfacts-mars.com/'
tables = pd.read_html(url2)
tables_df= tables[0]
tables_df=tables_df.rename(columns={0:'Description', 1:'Mars', 2:'Earth'})
tables_df.set_index('Description')
tables_df.to_html("Mars Facts.html")

## Mars Hemispheres

In [ ]:
url3= ['https://marshemispheres.com/cerberus.html', 'https://marshemispheres.com/schiaparelli.html', 'https://marshemispheres.com/syrtis.html', 'https://marshemispheres.com/valles.html']
titles_img=[]

for item in url3:
    browser.visit(item)
    html3 = browser.html

    news_soup3 = soup(html3, 'html.parser')
    
    slide_elem3 = news_soup3.select_one('img.wide-image')
    slide_elem3_url= item +'/'+ slide_elem3['src']
    slide_elem4 = news_soup3.select_one('div', class_='cover')
    list1=[slide_elem4.h2.text, slide_elem3_url]
    titles_img.append(list1)

hemisphere_image_url=[]
for title, img in titles_img:
    hemisphere_dict={'Title':title, 'img_url': img}
    hemisphere_image_url.append(hemisphere_dict)

In [ ]:
hemisphere_image_url

In [167]:
def scrape_all():
   # Initiate headless driver for deployment
   executable_path = {'executable_path': ChromeDriverManager().install()}
   browser = Browser('chrome', **executable_path, headless=True)
   news_title, news_paragraph = mars_news(browser)
   # Run all scraping functions and store results in a dictionary
   data = {
       "news_title": news_title,
       "news_paragraph": news_paragraph,
       "featured_image": featured_image(browser),
       "facts": mars_facts(),
       "hemispheres": mars_hemisphere(browser),
       "last_modified": dt.datetime.now()
   }
   # Stop webdriver and return data
   browser.quit()
   return data

def mars_news(browser):
   # Scrape Mars News
   # Visit the mars nasa news site
   url = 'https://redplanetscience.com/'
   browser.visit(url)
   # Optional delay for loading the page
   browser.is_element_present_by_css('div.list_text', wait_time=1)
   # Convert the browser html to a soup object and then quit the browser
   html = browser.html
   news_soup = soup(html, 'html.parser')
   # Add try/except for error handling
   try:
       slide_elem = news_soup.select_one('div.list_text')
       news_title = slide_elem.find("div", class_="content_title").get_text()
       news_p = slide_elem.find("div", class_="article_teaser_body").get_text()
   except AttributeError:
       return None, None
   return news_title, news_p

#Featured Image
def featured_image(browser):
    url1 = 'https://spaceimages-mars.com/'
    browser.visit(url1)
    html1 = browser.html
    news_soup1 = soup(html1, 'html.parser')
    try:
        slide_elem1 = news_soup1.select_one('div.floating_text_area')
        image_url_location = slide_elem1.find('a')
        image_url= image_url_location['href']
    except AttributeError:
        return None
    featured_image_url= (f'{url1}{image_url}')
    return(featured_image_url)


def mars_facts():
    #Mars Facts
    url2 = 'https://galaxyfacts-mars.com/'
    tables = pd.read_html(url2)
    tables_df= tables[0]
    tables_df=tables_df.rename(columns={0:'Description', 1:'Mars', 2:'Earth'})
    tables_df.set_index('Description')
    tables_df.to_html("Mars Facts.html")
    return(tables_df)


def mars_hemisphere(browser):
    #Mars Hemispheres
    url3= ['https://marshemispheres.com/cerberus.html', 'https://marshemispheres.com/schiaparelli.html', 'https://marshemispheres.com/syrtis.html', 'https://marshemispheres.com/valles.html']
    titles_img=[]

    for item in url3:
        browser.visit(item)
        html3 = browser.html

        news_soup3 = soup(html3, 'html.parser')
        try:
            slide_elem3 = news_soup3.select_one('img.wide-image')
            slide_elem3_url= 'https://marshemispheres.com/'+ slide_elem3['src']
            slide_elem4 = news_soup3.select_one('div', class_='cover')
        except AttributeError:
            return None
        list1=[slide_elem4.h2.text, slide_elem3_url]
        titles_img.append(list1)

    hemisphere_image_url=[]
    for title, img in titles_img:
        hemisphere_dict={'Title':title, 'img_url': img}
        hemisphere_image_url.append(hemisphere_dict)

    return(hemisphere_image_url)

In [168]:
mars_hemisphere(browser)

[{'Title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'Title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'Title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'Title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]

In [161]:
scrape_all()



====== WebDriver manager ======
Could not get version for google-chrome with the command:  powershell "$ErrorActionPreference='silentlycontinue' ; (Get-Item -Path "$env:PROGRAMFILES\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion ; if (-not $? -or $? -match $error) { (Get-Item -Path "$env:PROGRAMFILES(x86)\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion } if (-not $? -or $? -match $error) { (Get-Item -Path "$env:LOCALAPPDATA\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion } if (-not $? -or $? -match $error) { reg query "HKCU\SOFTWARE\Google\Chrome\BLBeacon" /v version } if (-not $? -or $? -match $error) { reg query "HKLM\SOFTWARE\Wow6432Node\Microsoft\Windows\CurrentVersion\Uninstall\Google Chrome" /v version }"
Current google-chrome version is UNKNOWN
Get LATEST chromedriver version for UNKNOWN google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/100.0.4896.60/chromedriver_win32.zip
Driver has been 

{'news_title': "Follow NASA's Perseverance Rover in Real Time on Its Way to Mars",
 'news_paragraph': "A crisply rendered web application can show you where the agency's Mars 2020 mission is right now as it makes its way to the Red Planet for a Feb. 18, 2021, landing.",
 'featured_image': 'https://spaceimages-mars.com/image/featured/mars1.jpg',
 'facts':                Description             Mars            Earth
 0  Mars - Earth Comparison             Mars            Earth
 1                Diameter:         6,779 km        12,742 km
 2                    Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 3                   Moons:                2                1
 4       Distance from Sun:   227,943,824 km   149,598,262 km
 5          Length of Year:   687 Earth days      365.24 days
 6             Temperature:     -87 to -5 °C      -88 to 58°C,
 'hemispheres': [{'Title': 'Cerberus Hemisphere Enhanced',
   'img_url': 'https://marshemispheres.com/cerberus.html/images/f5e372a36edfa389625da6d0